<a href="https://colab.research.google.com/github/Anou26/Projects/blob/main/DogeCoin_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

Preprocessing Data

Importing Dataset


In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/RohitGanji/dogecoin-price-prediction/main/DOGE-USD.csv",
                      parse_dates=["Date"])

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

Checking for null values

In [ ]:
dataset.isnull().sum()

Visualizing the data

In [ ]:
plt.figure(figsize=(27,7))
dataset.groupby('Date')['Closing Price (USD)'].median().plot(linewidth=3.5, color='r')
plt.xlabel('Date', fontsize=16)
plt.ylabel('Closing Price (USD)', fontsize=16)
plt.title("Date vs Closing Price (USD)", fontsize=18)
plt.grid(True);

In [ ]:
plt.figure(figsize=(27,7))
dataset.groupby('Date')['24h Open (USD)'].median().plot(linewidth = 3.5, color = 'y')
plt.xlabel('Date', fontsize=16)
plt.ylabel('24h Open (USD)', fontsize=16)
plt.title("Date vs 24h Open (USD)", fontsize=18)
plt.grid(True);

In [ ]:
plt.figure(figsize=(27,7))
dataset.groupby('Date')['24h High (USD)'].median().plot(linewidth=3.5, color='b')
plt.xlabel('Date', fontsize=16)
plt.ylabel('24h High (USD)', fontsize=16)
plt.title("Date vs 24h High (USD)", fontsize=18)
plt.grid(True);

In [ ]:
plt.figure(figsize=(27,7))
dataset.groupby('Date')['24h Low (USD)'].median().plot(linewidth=3.5, color='g')
plt.xlabel('Date', fontsize=16)
plt.ylabel('24h Low (USD)', fontsize=16)
plt.title("Date vs 24h Low (USD)", fontsize=18)
plt.grid(True);

In [ ]:
subset = pd.read_csv("https://raw.githubusercontent.com/RohitGanji/dogecoin-price-prediction/main/DOGE-USD.csv", parse_dates=["Date"], index_col="Date",
                      usecols=["Date", "Closing Price (USD)"])
subset.columns = ["Price"]

Generating a windowed dataset

In [ ]:
HORIZON = 1
WINDOW_SIZE = 7

windowed_dataset = subset.copy()

for i in range(WINDOW_SIZE):
  windowed_dataset[f"Price+{i+1}"] = windowed_dataset[f"Price"].shift(periods=i+1)

windowed_dataset.head(10)

In [ ]:
X = windowed_dataset.dropna().drop("Price", axis=1).astype(np.float32)
y = windowed_dataset.dropna()["Price"].astype(np.float32).to_frame()
print(X.shape, y.shape)

In [ ]:
X.head()

In [ ]:
y.head()

Splitting the dataset

In [ ]:
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size].values, y[:split_size].values
X_test, y_test = X[split_size:].values, y[split_size:].values
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(windowed_dataset.Price[:split_size], label="Train Data", linewidth=2.5)
plt.plot(windowed_dataset.Price[split_size-1:], label="Test Data", linewidth=2.5)
plt.xlabel("Time", fontsize=14)
plt.ylabel("DogeCoin Price (USD)", fontsize=14)
plt.title("Train Data & Test Data", fontsize=18);
plt.legend(fontsize=14)
plt.grid(True);

Modelling

In [ ]:
#Building and compiling the model
from tensorflow.keras import layers

model = tf.keras.Sequential([
      layers.Lambda(lambda x: tf.expand_dims(x, axis=1)),
      layers.Bidirectional(layers.LSTM(WINDOW_SIZE, return_sequences=True),
                           input_shape=(WINDOW_SIZE, X_train.shape[-1])),
      layers.Dropout(0.2),
      layers.Bidirectional(layers.LSTM(WINDOW_SIZE*2, return_sequences=True)),
      layers.Dense(HORIZON, activation="linear")
])

model.compile(loss="mse",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=["mae"])

In [ ]:
#Creating a callback
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor="val_loss",
                           patience=15,
                           restore_best_weights=True)]

In [ ]:
#Fitting the model
history = model.fit(X_train, y_train,
                    epochs=1000, batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
#visualizing the loss
epochs = list(range(1, len(history.epoch)+1))
plt.figure(figsize=(10,7))
plt.plot(epochs, history.history["loss"], label="Loss", linewidth=2.5)
plt.plot(epochs, history.history["val_loss"], label="Validaiton Loss", linewidth=2.5)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.title("Epoch vs Loss", fontsize=18);
plt.legend(fontsize=14)
plt.grid(True);

In [ ]:
#predictions
results = np.squeeze(model.predict(X_test))
preds = pd.DataFrame(results, y[split_size:].index)


In [ ]:
mae = tf.reduce_mean(tf.keras.metrics.mean_absolute_error(y_test, results)).numpy()
mse = tf.reduce_mean(tf.keras.metrics.mean_squared_error(y_test, results)).numpy()
mape = tf.reduce_mean(tf.keras.metrics.mean_absolute_percentage_error(y_test, results)).numpy()

print("mae:", mae)
print("mse:", mse)
print("rmse:", tf.sqrt(mse).numpy())
print("mape:", mape)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(y[split_size:], label="Test Data", linewidth=2.5)
# plt.plot(preds, label="Predicted Data", linewidth=2.5)
plt.xlabel("Time", fontsize=16)
plt.ylabel("DogeCoin Price", fontsize=16)
plt.title("Test Data vs Predicted Data", fontsize=18);
plt.legend(fontsize=14)
plt.grid(True);